In [ ]:



# Define directories and patterns
au_directory = '/home/nishma/My Project/College_thesis/IFD/archive/small_dataset/Au'
tp_directory = '/home/nishma/My Project/College_thesis/IFD/archive/small_dataset/Tp'
au_pattern = r'Au_([a-z]{3})_(\d{5})'
tp_pattern = r'([a-z]{3})(\d{5})_([a-z]{3})(\d{5})'
size = (128, 128)

# Process images
X, y = process_images(au_directory, tp_directory, au_pattern, tp_pattern, size=size)

# Output shape
print(X.shape, y.shape)



from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Load the MobileNetV2 model with pre-trained weights, excluding the top layers
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the convolutional base of the model to retain pre-trained weights during initial training
base_model.trainable = False

# Create the model architecture by adding custom layers on top of MobileNetV2
inputs = tf.keras.Input(shape=(128, 128, 3))
x = base_model(inputs, training=False)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
 loss='binary_crossentropy',
 metrics=['accuracy'])

# Summary of the model architecture
model.summary()

# Now you can proceed with the training and validation split
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

# Now you can use the `train_images` and `train_labels` in your model training code
history = model.fit(train_images, train_labels,
 validation_data=(test_images, test_labels),
 epochs=10)

In [ ]:
import os
import re
from PIL import Image, ImageFilter
import numpy as np
import cv2
import pathlib
import tensorflow as tf

In [ ]:
# Function to highlight the forged part in the image
def highlight_forgery(subtracted_array):
    gray_img = cv2.cvtColor(subtracted_array, cv2.COLOR_RGB2GRAY)
    _, thresholded_img = cv2.threshold(gray_img, 30, 255, cv2.THRESH_BINARY)
    edges = cv2.Canny(thresholded_img, 100, 200)
    highlighted_img = cv2.applyColorMap(edges, cv2.COLORMAP_JET)
    result = cv2.addWeighted(subtracted_array, 0.7, highlighted_img, 0.3, 0)
    return result

# Function to resize and subtract two images, and highlight the forgery
def subtract_images(image1_path, image2_path, size=(128, 128)):
 try:
 img1 = Image.open(image1_path).convert('RGB')
 img2 = Image.open(image2_path).convert('RGB')
 img1 = img1.resize(size)
 img2 = img2.resize(size)
 img1_array = np.asarray(img1, dtype=np.int32)
 img2_array = np.asarray(img2, dtype=np.int32)
 subtracted_array = np.clip(np.abs(img1_array - img2_array), 0, 255).astype(np.uint8)
 highlighted_array = highlight_forgery(subtracted_array)
 return highlighted_array
 except Exception as e:
 print(f"Error subtracting images: {e}")
 return None

# Function to find matching images by pattern
def find_image_pairs(au_directory, tp_directory, au_pattern, tp_pattern):
 authentic_images = {}
 tampered_images = []
 au_regex = re.compile(au_pattern)
 tp_regex = re.compile(tp_pattern)
 for filename in os.listdir(au_directory):
 if filename.endswith(('.jpg', '.jpeg', '.png', '.tiff')):
 match = au_regex.search(filename)
 if match:
 prefix = match.group(1)
 symbol = match.group(2)
 authentic_images[(prefix, symbol)] = os.path.join(au_directory, filename)
 for filename in os.listdir(tp_directory):
 if filename.endswith(('.jpg', '.jpeg', '.png', '.tiff', '.tif')):
 tampered_images.append(os.path.join(tp_directory, filename))
 return authentic_images, tampered_images

# Function to process matching images and append them to the dataset
def process_images(au_directory, tp_directory, au_pattern, tp_pattern, size=(128, 128)):
 authentic_images, tampered_images = find_image_pairs(au_directory, tp_directory, au_pattern, tp_pattern)
 X, y = [], []
 for tampered_img in tampered_images:
 tampered_filename = os.path.basename(tampered_img)
 print(tampered_filename)
 tampered_match = re.search(tp_pattern, tampered_filename)
 print(tampered_match)
 if tampered_match:
 tp_prefix1 = tampered_match.group(1)
 tp_symbol1 = tampered_match.group(2)
 tp_prefix2 = tampered_match.group(3)
 tp_symbol2 = tampered_match.group(4)
 matching_auth_img1 = authentic_images.get((tp_prefix1, tp_symbol1), None)
 matching_auth_img2 = authentic_images.get((tp_prefix2, tp_symbol2), None)
 print(matching_auth_img1)

 if matching_auth_img1:
 processed_img1 = subtract_images(matching_auth_img1, tampered_img, size=size)
 if processed_img1 is not None:
 X.append(processed_img1)
 y.append(1) # Label for forged

 if matching_auth_img2:
 processed_img2 = subtract_images(matching_auth_img2, tampered_img, size=size)
 if processed_img2 is not None:
 X.append(processed_img2)
 y.append(1) # Label for forged
 if not in matching_auth_img1 and matching_auth_img2:
 print(f"No matching authentic image found for tampered image: {tampered_filename}")
 return np.array(X), np.array(y)